In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import requests
import time

D:\Python\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
url = 'https://books.toscrape.com/catalogue/page-1.html'
driver = webdriver.Chrome()
driver.implicitly_wait(60)
driver.get(url)
soup = BeautifulSoup(driver.page_source,'xml')
driver.quit()

<?xml version="1.0" encoding="utf-8"?>
<html class="no-js" lang="en-us"><!--<![endif]--><head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=UTF-8" http-equiv="content-type">
<meta content="24th Jun 2016 09:30" name="created">
<meta content="" name="description">
<meta content="width=device-width" name="viewport">
<meta content="NOARCHIVE,NOCACHE" name="robots">
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../static/oscar/favicon.ico" rel="shortcut icon">
<link href="../static/oscar/css/styles.css" rel="stylesheet" type="text/css">
<link href="../static/oscar/js/bootstrap-datetimepicker/bootstrap-datetimepicker.css" rel="stylesheet">
<link href="../static/oscar/css/datetimepicker.css" rel="stylesheet" type="text/css">
</link>
<body class="default" id="default">
<header class="header container-flu

In [5]:
soup.find('h3')

<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>

In [6]:
soup.find('h3').string

'A Light in the ...'

In [8]:
[x.string for x in soup.find_all('h3')]

['A Light in the ...',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History ...',
 'The Requiem Red',
 'The Dirty Little Secrets ...',
 'The Coming Woman: A ...',
 'The Boys in the ...',
 'The Black Maria',
 'Starving Hearts (Triangular Trade ...',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little ...",
 'Rip it Up and ...',
 'Our Band Could Be ...',
 'Olio',
 'Mesaerion: The Best Science ...',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

In [9]:
[x.string for x in soup.find_all(attrs={'class':'price_color'})]

['£51.77',
 '£53.74',
 '£50.10',
 '£47.82',
 '£54.23',
 '£22.65',
 '£33.34',
 '£17.93',
 '£22.60',
 '£52.15',
 '£13.99',
 '£20.66',
 '£17.46',
 '£52.29',
 '£35.02',
 '£57.25',
 '£23.88',
 '£37.59',
 '£51.33',
 '£45.17']

In [11]:
[None if x.find(attrs={'class':'price_color'}) == None else 
x.find(attrs={'class':'price_color'}).string 
for x in soup.find_all(attrs={'class':'col-xs-6 col-md-3 col-lg-3'})]

[]

In [14]:
def scrape_page(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(30)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'lxml')
    driver.quit()
    df = pd.DataFrame(list(zip([None if x == None else x.string for x in soup.find_all('h3')],
                               [None if x.find(attrs={'class':'price_color'}) == None else x.find(attrs={'class':'price_color'}).string.replace('£','') for x in soup.find_all(attrs={'class':'col-xs-6 col-sm-4 col-md-3 col-lg-3'})],
                               [None if x.find(attrs={'class':'instock availability'}).text == None else x.find(attrs={'class':'instock availability'}).text.strip() for x in soup.find_all(attrs={'class':'col-xs-6 col-sm-4 col-md-3 col-lg-3'})],
                               [None if x.find(attrs={'class':re.compile(r'star-rating$')}).get('class') == None else x.find(attrs={'class':re.compile(r'star-rating$')}).get('class')[1] for x in soup.find_all(attrs={'class':'col-xs-6 col-sm-4 col-md-3 col-lg-3'})])),
                      columns=['product_name','price','availability','rating'])
    return df

scrape_page('https://books.toscrape.com/catalogue/page-1.html')

,product_name,price,availability,rating
0,A Light in the ...,51.77,In stock,Three
1,Tipping the Velvet,53.74,In stock,One
2,Soumission,50.10,In stock,One
3,Sharp Objects,47.82,In stock,Four
4,Sapiens: A Brief History ...,54.23,In stock,Five
5,The Requiem Red,22.65,In stock,One
6,The Dirty Little Secrets ...,33.34,In stock,Four
7,The Coming Woman: A ...,17.93,In stock,Three
8,The Boys in the ...,22.60,In stock,Four
9,The Black Maria,52.15,In stock,One


In [15]:
def scrape_multiple_pages(url,pages):
    #Input parameters of url and number of pages to scrape. Put {} in place of page number in url.
    page_number = list(range(pages))
    df = pd.DataFrame(columns=['product_name','price','availability','rating'])
    for i in range(len(page_number)): #Loops through each page number in url.
        if requests.get(url.format(i+1)).status_code == 200: #If the url returns an OK 200 reponse, scrape the page. 
            df_page = scrape_page(url.format(i+1))
            df = df.append(df_page)
            time.sleep(5) #Wait 5 seconds.
        else:
            break
    return df

scrape_multiple_pages('https://books.toscrape.com/catalogue/page-{}.html',pages=2)

,product_name,price,availability,rating
0,A Light in the ...,51.77,In stock,Three
1,Tipping the Velvet,53.74,In stock,One
2,Soumission,50.10,In stock,One
3,Sharp Objects,47.82,In stock,Four
4,Sapiens: A Brief History ...,54.23,In stock,Five
5,The Requiem Red,22.65,In stock,One
6,The Dirty Little Secrets ...,33.34,In stock,Four
7,The Coming Woman: A ...,17.93,In stock,Three
8,The Boys in the ...,22.60,In stock,Four
9,The Black Maria,52.15,In stock,One
